##Simulator

In [ ]:
import random
import pandas as pd

def dice_roll():
  return random.randint(1, 6)

def simulate_battle(annalis, iteration, white_army, black_army, return_winner):
  annalis.append([iteration, white_army, black_army])
  iteration += 1

  if white_army == 0 or black_army == 0:
    if return_winner == 'all_data':
      #return entire battle
      return annalis

    elif return_winner == 'only_winner':
      #print(white_army, black_army)
      #only return winner
      if white_army > black_army:
        return 1
      else:
        return 0

    elif return_winner == 'man_killed':
      return white_army - black_army
  
  #make rolls for white and black according to the quantity of their men
  white_rolls = []
  if white_army >= 3:
    #then make 3 rolls
    for _ in range(0, 3):
      white_rolls.append(dice_roll())
  elif white_army == 2:
    #then only 2 rolls
    for _ in range(0, 2):
      white_rolls.append(dice_roll())
  elif white_army == 1:
    #the only 1 roll
    for _ in range(0, 1):
      white_rolls.append(dice_roll())

  black_rolls = []
  if black_army >= 3:
    #then make 3 rolls
    for _ in range(0, 3):
      black_rolls.append(dice_roll())
  elif black_army == 2:
    #then only 2 rolls
    for _ in range(0, 2):
      black_rolls.append(dice_roll())
  elif black_army == 1:
    #the only 1 roll
    for _ in range(0, 1):
      black_rolls.append(dice_roll())

  #sort both of them
  white_rolls, black_rolls = sorted(white_rolls, reverse=True), sorted(black_rolls, reverse=True)
  #equal size
  minimum = min(len(white_rolls), len(black_rolls))
  white_rolls, black_rolls = white_rolls[:minimum], black_rolls[:minimum]

  #kill men
  #white = defender
  #black = attacker
  for fighter in range(0, 3):
    try:
      if white_rolls[fighter] >= black_rolls[fighter]:
        #kill black
        black_army -= 1
      else:
        white_army -= 1
    except:
      pass
  #print(str(white_army) + " defenders remaining", str(black_army) + " attackers remaining")
  return simulate_battle(annalis, iteration, white_army, black_army, return_winner)

simulate_battle(list(), 0, 10, 10, 'man_killed')

In [ ]:
#transform into dataset
battle = pd.DataFrame(simulate_battle(list(), 0, 10, 10, 'all_data'))
battle.columns = ["Iteration", "Defenders", "Attackers"]
battle

##White Remaining (full battles recorder)

In [ ]:
#for one simulation only record white remaining
all_battles = list()
for _ in range(0, 100):
  all_battles.append([x[1] for x in simulate_battle(list(), 0, 20, 80, 'all_data')])
pd.DataFrame(all_battles)

all_battles = pd.DataFrame(all_battles)
all_battles = all_battles.drop_duplicates()
all_battles = all_battles.transpose()
all_battles

In [ ]:
#graph all stocks
import matplotlib.pyplot as plt

plt.figure(figsize=(28, 14))
#for c in table.columns.values:
  #print(c)

fig = plt.plot(all_battles.index, all_battles, lw=1, alpha=1)

#remove axis labels
#plt.gca().axes.get_yaxis().set_visible(False)

In [ ]:
pd.DataFrame(all_battles).to_csv('all_battles.csv')

##Man Killed

In [ ]:
#man killed per simulation
man_killed = list()
for _ in range(0, 1000000):
  man_killed.append(simulate_battle(list(), 0, 20, 80, 'man_killed'))
#sorted(man_killed, reverse=True)

In [ ]:
#number of points after a treshold
man_killed_ = pd.DataFrame(sorted(man_killed, reverse=True))
man_killed_ = man_killed_.loc[man_killed_[0] >= -5]

#successful simulations where defender has won or lost, but attackers remained with less or equal 5
print(len(man_killed)-man_killed_.shape[0], man_killed_.shape[0]/len(man_killed))

In [ ]:
pd.DataFrame(man_killed).hist(bins=87, figsize=(15, 9))

##Win vs. Loss

In [ ]:
#defender wins vs. defender loss
defender = 0
attacker = 0
for _ in range(0, 100000):
  out = simulate_battle(list(), 0, 3, 3, 'only_winner')
  if out == 1:
    defender += 1
  else:
    attacker += 1

print(defender, attacker, defender/attacker)